##### 디렉토리 예상 구성도
```
now_directory
│
├───get_datasets.ipynb (현재 파일의 위치)
|
└───datasets
    │
    ├───behavior1 (입력)
    │   └───image_0.jpg
    │       image_1.jpg
    │       ...
    |       locations.csv
    │   
    ├───behavior2 (입력)
    │   └───image_0.jpg
    │       image_1.jpg
    │       ...
    |       locations.csv
    │   
    ├───behavior3 (입력)
    │   └───image_0.jpg
    │       image_1.jpg
    │       ...
    |       locations.csv
    │   
    ├───behavior4 (입력)
    │   └───image_0.jpg
    │       image_1.jpg
    │       ...
    |       locations.csv
    │   
    └───behavior5 (입력)
        └───image_0.jpg
            image_1.jpg
            ...
            locations.csv
       
```

- 아래 코드를 `실행(Shift + Enter)` 후 행동 명을 입력하시면 디렉토리가 생성되면서 카메라가 실행됩니다.
- 첫 실행일 경우 카메라 창이 바로 뜨지 않지만 열려있을겁니다.
- `클릭할때마다 사진이 저장됩니다.`
- 혹시 csv 파일은 사진 디렉토리에 마지막에 자동으로 생성이 되는데, 추가로 사진을 찍으실때는 csv 파일을 닫은 후 찍어주시길 바랍니다. 
- 종료를 원하시면 소문자 `q`를 입력하시면 됩니다.
- 카메라에 `양 손, 양 볼, 신체 표시(다리 부분은 조금 잘려도 됩니다)`가 나오실떄 찍어주시면 굉장히 감사하겠습니다.

##### 사진 목록
  1. None (서있기 자세 (준비자세) + 그냥 아무렇게나 편하게 있는 자세 등등, 다른 자세보다 많이 찍어주세요)
  2. Hand_None (손 준비 자세)
  3. rollfeet1 (왼 앞으로, 오른 발 땅)
  4. rollfeet2 (오른발 앞으로, 왼발 땅)
  5. rollhand1 (왼 팔 위로, 오른 팔 아래로)
  6. rollhand2 (오른 팔 위로,  팔 아래로)
  7. clap1 (박수 치는 자세에서 양 손을 떨어뜨려주세요)
  8. clap2 (박수 치는 자세에서 양 손을 붙여주세요)
  9. hooray (만세 자세)
  10. jumphooray (점프를 위해 무릎을 숙이는 자세)
  11. thumbsup_left (왼손 따봉, 오른손은 인식만 되게 해주세요)
  12. thumbsup_right (오른손 따봉, 왼손은 인식만 되게 해주세요)
  13. thumbsup_both (양 손 따봉)
  14. rock_left (왼손 바위)
  15. rock_right (오손 바위)
  16. paper_left (왼손 보자기)
  17. paper_right (오른손 보자기)
  18. scissors_left (왼손 가위)
  19. scissors_right (오른손 가위)
  20. cheek_left (왼손으로 왼볼 찍기)
  21. cheek_right (오른손으로 오른볼 찍기)


- 대근육 운동에 대해서는 개수의 변수처리가 필요하다고 하여 카운트를 위해 2개의 행동으로 나누었습니다.
- 손을 사용하는 경우 게임 이용자가 어느 손으로 행동을 할지 몰라서 양손 모두 학습하기 위해 나누었습니다.

#### code -> 아래 창 클릭 후 shift + enter -> 찍으실 포즈의 버튼 클릭 ex)thumbsup_right, scissors_left, clap1 -> 그 이후 실행, 창이 나오시면 클릭하여 찍기

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# 버튼 목록
button_names = ['None', 'Hand_None', 'cheek_left', 'cheek_right', 'clap1', 'clap2',
                'hooray', 'jumphooray', 'paper_left', 'paper_right', 'rock_left', 'rock_right',
                'scissors_left', 'scissors_right', 'scissors23_left', 'scissors23_right', 'rollfeet1', 'rollfeet2', 
                'rollhand1', 'rollhand2', 'thumbsup_left', 'thumbsup_right', 'thumbsup_both']

# 변수 a 초기값 설정
directory_name = 'xxxx'

# 버튼 생성
buttons = [widgets.Button(description=name) for name in button_names]

# 버튼 클릭 이벤트 핸들러 함수
def on_button_click(b):
    global directory_name
    directory_name = b.description
    with output:
        clear_output(wait=True)
        print(f"{b.description} 버튼이 클릭되었습니다!")
        print(f"현재 directory_name: {directory_name}")

# 버튼과 이벤트 핸들러 함수 연결
for button in buttons:
    button.on_click(on_button_click)

output = widgets.Output()

# 버튼을 2 * 4 그리드 형태로 배치
buttons_grid = widgets.GridBox(buttons, layout=widgets.Layout(grid_template_columns="repeat(4, 150px)"))

# 버튼과 결과를 노트북에 표시
display(buttons_grid)
display(output)

GridBox(children=(Button(description='None', style=ButtonStyle()), Button(description='Hand_None', style=Butto…

Output()

---

In [2]:
import os
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
import csv

import multimodel
from multimodel import MultiModelDetector


def create_directory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)


def save_image(img, directory_name, click_position):
    file_count = len(os.listdir(directory_name))
    filename = os.path.join(directory_name, f'image_{file_count}.jpg')
    # img = cv2.flip(img, 1)  # 좌우 반전
    cv2.imwrite(filename, img)
    print(f'Saved {filename} at position {click_position}')


def on_mouse_click(event, x, y, flags, param):
    global all_locations
    if event == cv2.EVENT_LBUTTONDOWN:
        click_position = (x, y)
        save_image(param['img'], param['directory_name'], click_position)
        locations = mmd.image_get_location(param['img'])
        all_locations.append(locations)


dataset_directory = os.path.join('datasets', directory_name)

# 디렉토리 생성 또는 확인
create_directory(dataset_directory)

mmd = MultiModelDetector('find_image')
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('No Video')
    exit(1)

all_locations = []
while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        print('Video has ended')
        break

    if img.shape[0] > 480:
        img = cv2.resize(img, None, fx=0.5, fy=0.5)

    showed_img = mmd.find_body(img)

    # 화면에 출력
    name = f'MediaPipe Hand and Face: {directory_name}'
    cv2.imshow(name, showed_img)

    # 마우스 클릭 이벤트 처리
    cv2.setMouseCallback(name, on_mouse_click, {'img': img, 'directory_name': dataset_directory})

    # 'q' 키를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print('사용자에 의해 동영상이 종료되었습니다.')
        break

# 비디오 캡처 자원 해제
cap.release()
cv2.destroyAllWindows()

csv_file_path = os.path.join(dataset_directory, 'locations.csv')
mode = 'a' if os.path.exists(csv_file_path) else 'w'
with open(csv_file_path, mode, newline='') as csvfile:
    writer = csv.writer(csvfile)
    if mode == 'w':
        # 파일이 존재하지 않으면 헤더를 쓴다
        writer.writerow(['Location' + str(i) for i in range(1, 92)])
    writer.writerows(all_locations)

Saved datasets\None\image_8.jpg at position (447, 278)
Saved datasets\None\image_9.jpg at position (447, 278)
Saved datasets\None\image_10.jpg at position (447, 278)
Saved datasets\None\image_11.jpg at position (447, 278)
Saved datasets\None\image_12.jpg at position (447, 278)
Saved datasets\None\image_13.jpg at position (447, 278)
Saved datasets\None\image_14.jpg at position (447, 278)
Saved datasets\None\image_15.jpg at position (447, 278)
Saved datasets\None\image_16.jpg at position (447, 278)
Saved datasets\None\image_17.jpg at position (447, 278)
Saved datasets\None\image_18.jpg at position (447, 278)
Saved datasets\None\image_19.jpg at position (447, 278)
Saved datasets\None\image_20.jpg at position (447, 278)
Saved datasets\None\image_21.jpg at position (447, 278)
Saved datasets\None\image_22.jpg at position (447, 278)
Saved datasets\None\image_23.jpg at position (447, 278)
Saved datasets\None\image_24.jpg at position (447, 278)
Saved datasets\None\image_25.jpg at position (447,